# This is a more polished notebook, with the main results and essential code only

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%config Completer.use_jedi = False

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import lightgbm as lgb


In [4]:
df = pd.read_csv('C:\\Users\\gmbas\\Downloads\\Files_lol_project\\datasets\\932_matches_df_21_06')
df

,Unnamed: 0,5th_CS_min_blue,4th_CS_min_blue,3rd_CS_min_blue,2nd_CS_min_blue,1st_CS_min_blue,5th_CS_min_red,4th_CS_min_red,3rd_CS_min_red,2nd_CS_min_red,...,4th_MP_blue,3rd_MP_blue,2nd_MP_blue,1st_MP_blue,5th_MP_red,4th_MP_red,3rd_MP_red,2nd_MP_red,1st_MP_red,target
0,0,1.976583,4.573580,4.756204,5.939718,6.626648,1.162788,4.747214,5.366882,5.895539,...,6045,27869,139033,90591,17226,60900,104796.0,5702,6894.0,1
1,1,3.867070,5.469181,5.623977,6.249752,6.870524,0.740881,3.587890,5.125936,5.649067,...,12382,39489,1173761,230329,2342839,25119,39085.0,120399,81033.0,0
2,2,1.080000,4.598162,4.825763,5.340403,5.681439,2.085663,4.498963,4.766707,5.405575,...,35375,18062,30808,53165,10990,1209,11605.0,17510,1377402.0,1
3,3,0.363769,2.671286,4.984266,5.148903,5.976233,1.734322,5.071400,5.178816,6.166968,...,191837,193851,27186,159593,464856,47094,58443.0,87329,19550.0,0
4,4,1.368294,4.925813,5.484073,5.915913,6.164462,0.815051,5.392639,5.477247,6.682713,...,101033,1233,34450,15688,11980,22250,52517.0,71398,11441.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,927,0.206679,3.242855,3.858181,4.125231,5.436693,3.042890,4.194576,4.323031,4.890833,...,10650,35904,56382,495855,66034,2743,709380.0,721154,202130.0,1
928,928,1.513932,4.858114,5.456084,6.261634,6.332916,4.486468,4.896937,4.932086,5.188566,...,1414,85240,100878,543559,6775,946303,12643.0,3374,2092.0,1
929,929,1.941900,4.367879,4.490651,5.920417,6.566390,0.475627,3.839915,4.261205,5.287674,...,7072,37538,139703,150471,152271,8974,27138.0,9033,178701.0,0
930,930,1.697442,2.560060,5.208572,6.171789,6.474445,1.276204,5.618855,6.072157,6.331045,...,3571,48540,11851,3029,38679,352602,394108.0,65706,825205.0,1


In [5]:
df.columns

Index(['Unnamed: 0', '5th_CS_min_blue', '4th_CS_min_blue', '3rd_CS_min_blue',
       '2nd_CS_min_blue', '1st_CS_min_blue', '5th_CS_min_red',
       '4th_CS_min_red', '3rd_CS_min_red', '2nd_CS_min_red', '1st_CS_min_red',
       '5th_kda_blue', '4th_kda_blue', '3rd_kda_blue', '2nd_kda_blue',
       '1st_kda_blue', '5th_kda_red', '4th_kda_red', '3rd_kda_red',
       '2nd_kda_red', '1st_kda_red', '5th_MP_blue', '4th_MP_blue',
       '3rd_MP_blue', '2nd_MP_blue', '1st_MP_blue', '5th_MP_red', '4th_MP_red',
       '3rd_MP_red', '2nd_MP_red', '1st_MP_red', 'target'],
      dtype='object')

In [6]:
df = df.drop('Unnamed: 0',axis = 1)
df

,5th_CS_min_blue,4th_CS_min_blue,3rd_CS_min_blue,2nd_CS_min_blue,1st_CS_min_blue,5th_CS_min_red,4th_CS_min_red,3rd_CS_min_red,2nd_CS_min_red,1st_CS_min_red,...,4th_MP_blue,3rd_MP_blue,2nd_MP_blue,1st_MP_blue,5th_MP_red,4th_MP_red,3rd_MP_red,2nd_MP_red,1st_MP_red,target
0,1.976583,4.573580,4.756204,5.939718,6.626648,1.162788,4.747214,5.366882,5.895539,6.067965,...,6045,27869,139033,90591,17226,60900,104796.0,5702,6894.0,1
1,3.867070,5.469181,5.623977,6.249752,6.870524,0.740881,3.587890,5.125936,5.649067,6.146694,...,12382,39489,1173761,230329,2342839,25119,39085.0,120399,81033.0,0
2,1.080000,4.598162,4.825763,5.340403,5.681439,2.085663,4.498963,4.766707,5.405575,6.289723,...,35375,18062,30808,53165,10990,1209,11605.0,17510,1377402.0,1
3,0.363769,2.671286,4.984266,5.148903,5.976233,1.734322,5.071400,5.178816,6.166968,6.529087,...,191837,193851,27186,159593,464856,47094,58443.0,87329,19550.0,0
4,1.368294,4.925813,5.484073,5.915913,6.164462,0.815051,5.392639,5.477247,6.682713,7.049121,...,101033,1233,34450,15688,11980,22250,52517.0,71398,11441.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,0.206679,3.242855,3.858181,4.125231,5.436693,3.042890,4.194576,4.323031,4.890833,5.414535,...,10650,35904,56382,495855,66034,2743,709380.0,721154,202130.0,1
928,1.513932,4.858114,5.456084,6.261634,6.332916,4.486468,4.896937,4.932086,5.188566,5.850610,...,1414,85240,100878,543559,6775,946303,12643.0,3374,2092.0,1
929,1.941900,4.367879,4.490651,5.920417,6.566390,0.475627,3.839915,4.261205,5.287674,5.424155,...,7072,37538,139703,150471,152271,8974,27138.0,9033,178701.0,0
930,1.697442,2.560060,5.208572,6.171789,6.474445,1.276204,5.618855,6.072157,6.331045,6.960445,...,3571,48540,11851,3029,38679,352602,394108.0,65706,825205.0,1


In [7]:
df = df.drop(['5th_MP_blue', '4th_MP_blue',
       '3rd_MP_blue', '2nd_MP_blue', '1st_MP_blue', '5th_MP_red', '4th_MP_red',
       '3rd_MP_red', '2nd_MP_red', '1st_MP_red'],axis = 1)
df

,5th_CS_min_blue,4th_CS_min_blue,3rd_CS_min_blue,2nd_CS_min_blue,1st_CS_min_blue,5th_CS_min_red,4th_CS_min_red,3rd_CS_min_red,2nd_CS_min_red,1st_CS_min_red,...,4th_kda_blue,3rd_kda_blue,2nd_kda_blue,1st_kda_blue,5th_kda_red,4th_kda_red,3rd_kda_red,2nd_kda_red,1st_kda_red,target
0,1.976583,4.573580,4.756204,5.939718,6.626648,1.162788,4.747214,5.366882,5.895539,6.067965,...,2.250000,2.516667,2.750000,3.111111,1.354167,1.857143,1.916667,2.000000,2.316667,1
1,3.867070,5.469181,5.623977,6.249752,6.870524,0.740881,3.587890,5.125936,5.649067,6.146694,...,1.261364,1.571429,2.181818,2.950000,1.520833,1.777778,2.000000,2.173611,3.614286,0
2,1.080000,4.598162,4.825763,5.340403,5.681439,2.085663,4.498963,4.766707,5.405575,6.289723,...,1.472222,1.645833,1.816667,2.160714,1.708333,1.895833,2.071429,2.386364,2.833333,1
3,0.363769,2.671286,4.984266,5.148903,5.976233,1.734322,5.071400,5.178816,6.166968,6.529087,...,1.544643,2.275000,2.750000,3.100000,1.623932,1.950000,2.171429,2.616667,2.801587,0
4,1.368294,4.925813,5.484073,5.915913,6.164462,0.815051,5.392639,5.477247,6.682713,7.049121,...,2.225000,2.330357,2.937500,3.071429,1.866071,1.900000,2.183333,2.750000,4.958333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,0.206679,3.242855,3.858181,4.125231,5.436693,3.042890,4.194576,4.323031,4.890833,5.414535,...,1.812500,2.314286,2.439394,2.500000,1.458333,1.466667,1.483333,1.680556,2.048611,1
928,1.513932,4.858114,5.456084,6.261634,6.332916,4.486468,4.896937,4.932086,5.188566,5.850610,...,2.279221,2.550000,2.687500,2.928571,1.107955,1.236111,1.662500,1.755556,1.812500,1
929,1.941900,4.367879,4.490651,5.920417,6.566390,0.475627,3.839915,4.261205,5.287674,5.424155,...,1.928571,1.944444,2.303030,2.635714,1.850000,3.000000,3.083333,3.125000,3.162338,0
930,1.697442,2.560060,5.208572,6.171789,6.474445,1.276204,5.618855,6.072157,6.331045,6.960445,...,1.950000,2.100000,2.900000,3.660714,1.633333,1.708333,2.083333,2.675000,3.000000,1


In [8]:
df.columns

Index(['5th_CS_min_blue', '4th_CS_min_blue', '3rd_CS_min_blue',
       '2nd_CS_min_blue', '1st_CS_min_blue', '5th_CS_min_red',
       '4th_CS_min_red', '3rd_CS_min_red', '2nd_CS_min_red', '1st_CS_min_red',
       '5th_kda_blue', '4th_kda_blue', '3rd_kda_blue', '2nd_kda_blue',
       '1st_kda_blue', '5th_kda_red', '4th_kda_red', '3rd_kda_red',
       '2nd_kda_red', '1st_kda_red', 'target'],
      dtype='object')

In [9]:
df = df.dropna()

In [10]:
df

,5th_CS_min_blue,4th_CS_min_blue,3rd_CS_min_blue,2nd_CS_min_blue,1st_CS_min_blue,5th_CS_min_red,4th_CS_min_red,3rd_CS_min_red,2nd_CS_min_red,1st_CS_min_red,...,4th_kda_blue,3rd_kda_blue,2nd_kda_blue,1st_kda_blue,5th_kda_red,4th_kda_red,3rd_kda_red,2nd_kda_red,1st_kda_red,target
0,1.976583,4.573580,4.756204,5.939718,6.626648,1.162788,4.747214,5.366882,5.895539,6.067965,...,2.250000,2.516667,2.750000,3.111111,1.354167,1.857143,1.916667,2.000000,2.316667,1
1,3.867070,5.469181,5.623977,6.249752,6.870524,0.740881,3.587890,5.125936,5.649067,6.146694,...,1.261364,1.571429,2.181818,2.950000,1.520833,1.777778,2.000000,2.173611,3.614286,0
2,1.080000,4.598162,4.825763,5.340403,5.681439,2.085663,4.498963,4.766707,5.405575,6.289723,...,1.472222,1.645833,1.816667,2.160714,1.708333,1.895833,2.071429,2.386364,2.833333,1
3,0.363769,2.671286,4.984266,5.148903,5.976233,1.734322,5.071400,5.178816,6.166968,6.529087,...,1.544643,2.275000,2.750000,3.100000,1.623932,1.950000,2.171429,2.616667,2.801587,0
4,1.368294,4.925813,5.484073,5.915913,6.164462,0.815051,5.392639,5.477247,6.682713,7.049121,...,2.225000,2.330357,2.937500,3.071429,1.866071,1.900000,2.183333,2.750000,4.958333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,0.206679,3.242855,3.858181,4.125231,5.436693,3.042890,4.194576,4.323031,4.890833,5.414535,...,1.812500,2.314286,2.439394,2.500000,1.458333,1.466667,1.483333,1.680556,2.048611,1
928,1.513932,4.858114,5.456084,6.261634,6.332916,4.486468,4.896937,4.932086,5.188566,5.850610,...,2.279221,2.550000,2.687500,2.928571,1.107955,1.236111,1.662500,1.755556,1.812500,1
929,1.941900,4.367879,4.490651,5.920417,6.566390,0.475627,3.839915,4.261205,5.287674,5.424155,...,1.928571,1.944444,2.303030,2.635714,1.850000,3.000000,3.083333,3.125000,3.162338,0
930,1.697442,2.560060,5.208572,6.171789,6.474445,1.276204,5.618855,6.072157,6.331045,6.960445,...,1.950000,2.100000,2.900000,3.660714,1.633333,1.708333,2.083333,2.675000,3.000000,1


## Dividing in train and test sets, maintaining the order of the matches(match 0 was the oldest match played, while the last line was the most recent match).
## We can't randomly split the matches with let's say train_test_split due to data leakage possibility

In [11]:
y_all = df['target']
X_all = df.drop('target',axis = 1)

In [12]:
cutoff1 = round(df.shape[0]*0.8)
X,X_test = X_all[0:cutoff1],X_all[cutoff1:]
y,y_test = y_all[0:cutoff1],y_all[cutoff1:]

In [13]:
df

,5th_CS_min_blue,4th_CS_min_blue,3rd_CS_min_blue,2nd_CS_min_blue,1st_CS_min_blue,5th_CS_min_red,4th_CS_min_red,3rd_CS_min_red,2nd_CS_min_red,1st_CS_min_red,...,4th_kda_blue,3rd_kda_blue,2nd_kda_blue,1st_kda_blue,5th_kda_red,4th_kda_red,3rd_kda_red,2nd_kda_red,1st_kda_red,target
0,1.976583,4.573580,4.756204,5.939718,6.626648,1.162788,4.747214,5.366882,5.895539,6.067965,...,2.250000,2.516667,2.750000,3.111111,1.354167,1.857143,1.916667,2.000000,2.316667,1
1,3.867070,5.469181,5.623977,6.249752,6.870524,0.740881,3.587890,5.125936,5.649067,6.146694,...,1.261364,1.571429,2.181818,2.950000,1.520833,1.777778,2.000000,2.173611,3.614286,0
2,1.080000,4.598162,4.825763,5.340403,5.681439,2.085663,4.498963,4.766707,5.405575,6.289723,...,1.472222,1.645833,1.816667,2.160714,1.708333,1.895833,2.071429,2.386364,2.833333,1
3,0.363769,2.671286,4.984266,5.148903,5.976233,1.734322,5.071400,5.178816,6.166968,6.529087,...,1.544643,2.275000,2.750000,3.100000,1.623932,1.950000,2.171429,2.616667,2.801587,0
4,1.368294,4.925813,5.484073,5.915913,6.164462,0.815051,5.392639,5.477247,6.682713,7.049121,...,2.225000,2.330357,2.937500,3.071429,1.866071,1.900000,2.183333,2.750000,4.958333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,0.206679,3.242855,3.858181,4.125231,5.436693,3.042890,4.194576,4.323031,4.890833,5.414535,...,1.812500,2.314286,2.439394,2.500000,1.458333,1.466667,1.483333,1.680556,2.048611,1
928,1.513932,4.858114,5.456084,6.261634,6.332916,4.486468,4.896937,4.932086,5.188566,5.850610,...,2.279221,2.550000,2.687500,2.928571,1.107955,1.236111,1.662500,1.755556,1.812500,1
929,1.941900,4.367879,4.490651,5.920417,6.566390,0.475627,3.839915,4.261205,5.287674,5.424155,...,1.928571,1.944444,2.303030,2.635714,1.850000,3.000000,3.083333,3.125000,3.162338,0
930,1.697442,2.560060,5.208572,6.171789,6.474445,1.276204,5.618855,6.072157,6.331045,6.960445,...,1.950000,2.100000,2.900000,3.660714,1.633333,1.708333,2.083333,2.675000,3.000000,1


## Function to calculate the standard error of the mean, which will be used to calculate the accuracy's uncertainty

In [14]:
def acc_std_dev(mean_acc, N, y_pred, y_true):
    '''
    mean_acc: number from 0 to 1 that is the fraction of accurate predicts of the model given X_test as input
    N: number of lines in y_test
    y_pred: predicts done by the model given X_test as input
    y_true: It's y_test
    
    '''
    sum_ = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            y_i = 1
        else:
            y_i = 0
            
        sum_ += (y_i - mean_acc)**2
        
    std_dev = np.sqrt(sum_/(N-1))
    
    avg_std_dev = std_dev/np.sqrt(N)
    
    return avg_std_dev

# Logistic Regression

In [15]:
tscv = TimeSeriesSplit()

This specific choice of hyperparameters were taken from https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/

In [16]:
param_grid_log_reg = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
'penalty': ['none', 'l1', 'l2', 'elasticnet'],
'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]}

In [17]:
log_reg = LogisticRegression(random_state=42)

## Now we run gridsearch using the time series split for crossvalidation 

In [18]:
gs_log_reg = GridSearchCV(estimator=log_reg,
                  param_grid=param_grid_log_reg, 
                  scoring = 'accuracy',
                  n_jobs = -1,
                  cv = tscv.split(X))

In [19]:
results_gs_log_reg = gs_log_reg.fit(X,y)

C:\Users\gmbas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\gmbas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\gmbas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\gmbas\AppData\Local\Programs\Py

## Optimal Logistic regression hyperparameters found in cross validation:

In [20]:
print(f'Best Score: {results_gs_log_reg.best_score_}')
print(f'Best Hyperparameters:{results_gs_log_reg.best_params_}')

Best Score: 0.5371900826446281
Best Hyperparameters:{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}


In [21]:
hyperparams_logreg = results_gs_log_reg.best_params_

## Now with the optimal hyperparameters, we run the model with the entire train set using the optimal hyperparameters

In [23]:
log_reg_final = LogisticRegression(
                                   random_state = 42,
                                   **hyperparams_logreg
)

In [24]:
log_reg_final.fit(X,y)

LogisticRegression(C=0.1, random_state=42, solver='newton-cg')

In [25]:
log_reg_final.score(X,y)  #train score

0.5708390646492435

In [26]:
log_reg_score = log_reg_final.score(X_test,y_test)  #test score
log_reg_score

0.5054945054945055

## Now we calculate the accuracy's std error

In [27]:
y_pred_log_reg = log_reg_final.predict(X_test)
y_test_np_arr = np.array(y_test)
N = y_test.shape[0]

In [28]:
acc_std_dev(log_reg_score, N , y_pred_log_reg, y_test_np_arr)

0.03716246327001032

# Random forest

In [29]:
rf = RandomForestClassifier(random_state = 42)

In [30]:
param_grid_rf = {
    "n_estimators":[500],
    "max_features":[None,'sqrt'],
    "min_samples_leaf":np.arange(20,500,10)
    
    
    
}

## Now we run gridsearch using the time series split for crossvalidation 

In [31]:
gs_rf = GridSearchCV(estimator=rf,
                  param_grid=param_grid_rf, 
                  scoring = 'accuracy',
                  n_jobs = -1,
                  cv = tscv.split(X))

In [32]:
results_gs_rf = gs_rf.fit(X,y)

## Optimal Random forest hyperparameters found in cross validation:

In [33]:
print(f'Best Score: {results_gs_rf.best_score_}')
print(f'Best Hyperparameters:{results_gs_rf.best_params_}')

Best Score: 0.5305785123966943
Best Hyperparameters:{'max_features': None, 'min_samples_leaf': 40, 'n_estimators': 500}


In [34]:
hyperparams_rf = results_gs_rf.best_params_

## Now with the optimal hyperparameters, we run the model with the entire train set using the optimal hyperparameters

In [35]:
rf_final = RandomForestClassifier(
                                  random_state = 42,
                                  **hyperparams_rf
)

In [36]:
rf_final.fit(X,y)  

RandomForestClassifier(max_features=None, min_samples_leaf=40, n_estimators=500,
                       random_state=42)

In [37]:
rf_final.score(X,y)  #train score

0.7111416781292985

In [38]:
rf_score = rf_final.score(X_test,y_test)  #test score
rf_score

0.5604395604395604

## Now we calculate the accuracy's std error

In [39]:
y_pred_rf = rf_final.predict(X_test)

In [40]:
acc_std_dev(rf_score, N , y_pred_rf, y_test_np_arr)

0.036892187222875475

# Lightgbm

In [45]:
lgbm = lgb.LGBMClassifier(random_state = 42)

In [46]:
param_grid_lgbm = {
    'num_leaves':np.arange(1,2001,50),
    'min_data_in_leaf':np.arange(1,2001,50)
    #'max_depth':
}

In [47]:
gs_lgbm = GridSearchCV(estimator=lgbm,
                  param_grid=param_grid_lgbm, 
                  scoring = 'accuracy',
                  n_jobs = -1,
                  cv = tscv.split(X))

In [48]:
results_gs_lgbm = gs_lgbm.fit(X,y)

C:\Users\gmbas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 8000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\gmbas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\gmbas\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "C:\Users\gmbas\AppData

[LightGBM] [Warning] min_data_in_leaf is set=51, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=51


In [49]:
print(f'Best Score: {results_gs_lgbm.best_score_}')
print(f'Best Hyperparameters:{results_gs_lgbm.best_params_}')

Best Score: 0.5272727272727273
Best Hyperparameters:{'min_data_in_leaf': 51, 'num_leaves': 51}


In [50]:
hyperparams_lgbm = results_gs_lgbm.best_params_

In [51]:
lgbm_final = lgb.LGBMClassifier(**hyperparams_lgbm)

In [52]:
lgbm_final.fit(X,y)

LGBMClassifier(min_data_in_leaf=51, num_leaves=51)

In [53]:
lgbm_final.score(X,y) #train score

0.9876203576341128

In [54]:
lgbm_score = lgbm_final.score(X_test,y_test) #test score  
lgbm_score

0.46153846153846156

In [55]:
y_pred_lgbm = lgbm_final.predict(X_test)

In [56]:
acc_std_dev(lgbm_score, N , y_pred_lgbm, y_test_np_arr)

0.03705458941901795

# KNN

In [95]:
knn = KNeighborsClassifier(n_jobs = -1)

In [107]:
param_grid_knn = {
    'n_neighbors': np.arange(20,160,4),
    'weights' : ['uniform', 'distance'],
    #'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size' : np.arange(50,1000,50),
    'metric' : ['euclidean','manhattan','chebyshev','minkowski']
}

In [108]:
gs_knn = GridSearchCV(estimator=knn,
                  param_grid=param_grid_knn, 
                  scoring = 'accuracy',
                  n_jobs = -1,
                  cv = tscv.split(X))

In [109]:
results_gs_knn = gs_knn.fit(X,y)

C:\Users\gmbas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.46280992 0.47768595 0.48595041 ...        nan        nan        nan]
  warnings.warn(


In [110]:
print(f'Best Score: {results_gs_knn.best_score_}')
print(f'Best Hyperparameters:{results_gs_knn.best_params_}')

Best Score: 0.5024793388429751
Best Hyperparameters:{'leaf_size': 50, 'metric': 'manhattan', 'n_neighbors': 20, 'weights': 'uniform'}


In [111]:
hyperparams_knn = results_gs_knn.best_params_

In [112]:
knn_final = KNeighborsClassifier(
    **hyperparams_knn
#                               leaf_size = 1, 
#                               metric = 'manhattan', 
#                               n_neighbors = 50,
#                               weights = 'uniform'
                                  )

In [113]:
knn_final.fit(X,y)  

KNeighborsClassifier(leaf_size=50, metric='manhattan', n_neighbors=20)

In [114]:
knn_final.score(X,y)  #train score

0.6286107290233838

In [115]:
knn_score = knn_final.score(X_test,y_test)  #test score
knn_score

0.510989010989011

In [116]:
y_pred_rf = knn_final.predict(X_test)

In [117]:
acc_std_dev(knn_score, N , y_pred_rf, y_test_np_arr)

0.03715573032978477

# SVM

In [69]:
svm =SVC(random_state=42)

In [70]:
param_grid_svm = {
    'C': [1e-5,1e-3,  1e-1, 10],
    #'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
    'kernel':['linear']
    #'kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],

}

In [71]:
gs_svm = GridSearchCV(estimator=svm,
                  param_grid=param_grid_svm, 
                  scoring = 'accuracy',
                  n_jobs = -1,
                  cv = tscv.split(X))

In [72]:
results_gs_svm = gs_svm.fit(X,y)

In [73]:
print(f'Best Score: {results_gs_svm.best_score_}')
print(f'Best Hyperparameters:{results_gs_svm.best_params_}')

Best Score: 0.5107438016528926
Best Hyperparameters:{'C': 0.1, 'kernel': 'linear'}


In [74]:
hyperparams_svm = results_gs_svm.best_params_

In [75]:
svm_final = SVC(
                random_state = 42,
                **hyperparams_svm
                                  )

In [76]:
svm_final.fit(X,y)  

SVC(C=0.1, kernel='linear', random_state=42)

In [77]:
svm_final.score(X,y)  #train score

0.5845942228335625

In [78]:
svm_score = svm_final.score(X_test,y_test)  #test score
svm_score

0.5384615384615384

In [79]:
y_pred_rf = svm_final.predict(X_test)

In [80]:
acc_std_dev(svm_score, N , y_pred_rf, y_test_np_arr)

0.037054589419017965